<a href="https://colab.research.google.com/github/sowmen/Deepfake-Challenge-Kaggle/blob/master/FWA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!git clone "https://github.com/danmohaha/DSP-FWA.git"

Cloning into 'DSP-FWA'...
remote: Enumerating objects: 82, done.
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82
Unpacking objects: 100% (82/82), done.


In [0]:
import os, sys
sys.path.insert(1,'DSP-FWA/')

In [4]:
!pip install -U opencv-python==3.4.9.31

     |████████████████████████████████| 28.2MB 104kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [0]:
"""
Dual Spatial Pyramid for exposing face warp artifacts in DeepFake videos (DSP-FWA)
"""
import torch
import torch.nn.functional as F
import cv2, os, dlib
import numpy as np
from py_utils.face_utils import lib
from py_utils.vid_utils import proc_vid as pv
from py_utils.DL.pytorch_utils.models.classifier import SPPNet

In [0]:
sample_num = 10
# Employ dlib to extract face area and landmark points
# pwd = os.path.dirname(os.path.abspath(__file__))
front_face_detector = dlib.get_frontal_face_detector()
lmark_predictor = dlib.shape_predictor('DSP-FWA/dlib_model/shape_predictor_68_face_landmarks.dat')

In [0]:
def im_test(net, im):
    face_info = lib.align(im[:, :, (2,1,0)], front_face_detector, lmark_predictor)
    # Samples
    if len(face_info) != 1:
        prob = -1
    else:
        _, point = face_info[0]
        rois = []
        for i in range(sample_num):
            roi, _ = lib.cut_head([im], point, i)
            rois.append(cv2.resize(roi[0], (224, 224)))

        vis_ = np.concatenate(rois, 1)
        cv2.imwrite('vis.jpg', vis_)

        bgr_mean = np.array([103.939, 116.779, 123.68])
        bgr_mean = bgr_mean[np.newaxis, :, np.newaxis, np.newaxis]
        bgr_mean = torch.from_numpy(bgr_mean).float().cuda()

        rois = torch.from_numpy(np.array(rois)).float().cuda()
        rois = rois.permute((0, 3, 1, 2))
        prob = net(rois - bgr_mean)
        prob = F.softmax(prob, dim=1)
        prob = prob.data.cpu().numpy()
        prob = 1 - np.mean(np.sort(prob[:, 0])[np.round(sample_num / 2).astype(int):])
    return prob, face_info

In [0]:
def draw_face_score(im, face_info, prob):
    if len(face_info) == 0:
        return im

    _, points = np.array(face_info[0])
    x1 = np.min(points[:, 0])
    x2 = np.max(points[:, 0])
    y1 = np.min(points[:, 1])
    y2 = np.max(points[:, 1])

    # Real: (0, 255, 0), Fake: (0, 0, 255)
    color = (0, prob * 255, (1 - prob) * 255)
    cv2.rectangle(im, (x1, y1), (x2, y2), color, 10)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(im, '{:.3f}'.format(prob), (x1, y1 - 10), font, 1, color, 3, cv2.LINE_AA)
    return im

In [0]:
def setup():
    num_class = 2
    net = SPPNet(backbone=50, num_class=num_class)
    net = net.cuda()
    net.eval()
    return net

In [10]:
net = setup()
model_path = "drive/My Drive/ResNet50/SPP-res50.pth"
if os.path.isfile(model_path):
    print("=> loading checkpoint '{}'".format(model_path))
    checkpoint = torch.load(model_path)
    start_epoch = checkpoint['epoch']
    net.load_state_dict(checkpoint['net'])
    print("=> loaded checkpoint '{}' (epoch {})"
            .format(model_path, start_epoch))
else:
    raise ValueError("=> no checkpoint found at '{}'".format(model_path))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth



=> loading checkpoint 'drive/My Drive/ResNet50/SPP-res50.pth'
=> loaded checkpoint 'drive/My Drive/ResNet50/SPP-res50.pth' (epoch 49)


In [0]:
import time
TEST_DIR = "drive/My Drive/test2"

In [12]:
f_path = os.path.join(TEST_DIR, "aagfhgtpmv_FAKE.mp4")
print('Testing: ' + f_path)

start_time = time.time()
suffix = f_path.split('.')[-1]
if suffix.lower() in ['jpg', 'png', 'jpeg', 'bmp', 'tif', 'nef', 'raf']:
    im = cv2.imread(f_path)
    if im is None:
        prob = -1
    else:
        prob, face_info = im_test(net, im, args)
    print(prob)

elif suffix.lower() in ['mp4', 'avi', 'mov']:
    # Parse video
    imgs, frame_num, fps, width, height = pv.parse_vid(f_path)
    probs = []
    for fid, im in enumerate(imgs):
        print('Frame: ' + str(fid))
        prob, face_info = im_test(net, im)
        probs.append(prob)

    print(probs)

    elapsed = time.time() - start_time
    print("Elapsed %f sec" % (elapsed))

Testing: drive/My Drive/test2/aagfhgtpmv_FAKE.mp4
Frame: 0


RuntimeError: ignored